# Tutorial Notebook for nisarhdf.nisarRSLCHDF
---

This notebook demonstrates the functionality of `nisarRSLCHDF`, which is at tool for reading NISAR **SLC** products.  Some of its functionality is specifically developed for the Greenland Mapping Project (GrIMP) workflows, but it may also be of general use for working with these products. If nothing else, the source code provides examples of how to unpack these projects.

Note, this class largely serves to unpack and reformat RSLC data. As such, at present, there is no support for data cubes. The **RSLC** products can be large, so it may not work well on machines with limited memory.

## Installation

The easiest way to ensure the code works is to create a custom environment using the *environment.yml* file included in this *nisarhdf* repository. To do so execute the following,

`conda env create -f environment.yml`

Then setup the kernel for the Jupyter notebooks with the command:

`python -m ipykernel install --user --name=nisarcryo`

Then activate the environment:

`conda activate nisarcryo`

Finally, start Jupyter and open the notebooks.

**Even if you want to work with your existing environment, the *environment.yml* file demonstrates which packages must be installed if they are not already present.**


## Python Imports

In [1]:
%load_ext autoreload
%autoreload 2
import nisarhdf
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rioxarray

## Open the HDF

Edit the path below to point to the ROFF product you wish to work with.

In [2]:
RSLCFile = '/Volumes/insar1/ian/NISAR/strips/frame_280/rslc_products/rslc_ALOS2496725280-230804-008.h5'

Create an instance and open the hdf as shown below. For normal products, you shouldn't have to specify the orbit, but while the orbit fields are present in the example file, they are not populated. When you open the hdf, you can specify the precise orbit XMLs if desired, which are release a few weeks after the MOI orbits used in the processing. Note the **RSLC** files are large, so use `noLoadData=True` to just load the meta data.

In [3]:
myRSLC = nisarhdf.nisarRSLCHDF()
myRSLC.openHDF(RSLCFile, referenceOrbitXML=None, referenceOrbit=49672, frame=5280, noLoadData=False)

Could not open with page_buf_size, opening for non-optimized access
RSLC zeroDopplerStartTime
[660125.36827613]


## RSLC Parameters

A summary of some of the more important parameters for the **RUNW** products can be obtained by.

In [4]:
myRSLC.printParams()

RSLC.referenceOrbit = 49672 
RSLC.frame = 5280 
RSLC.datetime = 2023-08-04 14:57:47.166045 
RSLC.referenceGranule = None
RSLC.SLCNearRange = 814637.0 
RSLC.SLCFarRange = 852284.282920905 
RSLC.SLCFirstZeroDopplerTime = 53867.166045 
RSLC.SLCLastZeroDopplerTime = 53877.165933569675 
RSLC.LookDirection = right 
RSLC.PassType = descending 
RSLC.Wavelength = None
RSLC.PRF = 3051.234 
RSLC.epsg = None 
RSLC.SLCRangeSize = 26320 
RSLC.SLCAzimuthSize = 30513 
RSLC.SLCRangePixelSize = 1.4304222394811763 
RSLC.SLCAzimuthPixelSize = 2.2716394524093184 
RSLC.SLCZeroDopplerTimeSpacing = 0.0003277362535944474 
RSLC.SLCNearRange = 814637.0 
RSLC.SLCFirstZeroDopplerTime = 53867.166045 
RSLC.SLCIncidenceCenter = None

Data Fields
RSLC.HH


## Save RSLC to Binary File

The RSLC data can be dumped to a float binary (`np.complex64`) file with the following call:

In [ ]:
myRSLC.writeData(f'output/RSLC.{myRSLC.referenceOrbit}_{myRSLC.frame}', bands=['HH'], tiff=False)